## Apply PCA for a dataset given in the url, build a classification model and plot the graph

### Data Preprocessing

In [ ]:
import numpy as np
import pandas as pd
import sys
sys.path.append('/home/admin1/PycharmProjects/Machine Learning using libraries/')
import os
os.chdir('/home/admin1/PycharmProjects/Machine Learning using libraries/Classification/Datasets & pickled objects/')
from ipynb.fs.full.ml_library import *

# reading dataset from file & storing it as pandas dataframe
wine_data = pd.read_csv('Wine.csv')
wine_data.head()

In [ ]:
wine_data.info()       # observing datatypes of different columns & checking null values

#### Separating out feature colums & label column 

In [ ]:
x_values = wine_data.iloc[:,:-1].values
y_values = wine_data.iloc[:,-1].values

#### Splitting dataset into train set & test set

In [ ]:
from sklearn.model_selection import train_test_split
train_x_values, test_x_values, train_y_values, test_y_values = train_test_split(x_values, y_values, train_size=0.8, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler

sc_x = StandardScaler()
train_x_values = sc_x.fit_transform(train_x_values)
test_x_values = sc_x.transform(test_x_values)

#### Applying Principal Component Analysis (PCA)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=None)
train_x_values = pca.fit_transform(train_x_values)
test_x_values = pca.transform(test_x_values)
explained_variance = pca.explained_variance_ratio_
explained_variance

In [ ]:
pca = PCA(n_components=2)
train_x_values = pca.fit_transform(train_x_values)
test_x_values = pca.transform(test_x_values)
explained_variance = pca.explained_variance_ratio_
explained_variance

#### observed 2 components are enough to describe sufficient variance 

### Building Support Vector Machine (SVM) model

In [ ]:
from sklearn.svm import SVC

classifier = SVC(kernel='rbf',random_state=0, probability=True)
classifier.fit(train_x_values, train_y_values)

#### Storing essential objects into pickle file

In [1]:
import joblib
file = open('StdScalingWine.pkl', 'wb')
joblib.dump(sc_x, file)
file.close()

file = open('PCAWine.pkl', 'wb')
joblib.dump(pca, file)
file.close()

file = open('PCASVMModelWine.pkl', 'wb')
joblib.dump(classifier, file)
file.close()

NameError: name 'sc_x' is not defined

#### Storing predictions for test set

In [ ]:
test_prediction = classifier.predict(test_x_values)

### Evaluating model against test set 

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
print(f'Confusion matrix:\n {confusion_matrix(test_y_values, test_prediction)}')
print(f'\nAccuracy score: {accuracy_score(test_y_values, test_prediction)}')
print(f'\nClassification report:\n {classification_report(test_y_values, test_prediction)}')

### Plotting colormap for  classification

#### 1, 2 & 3 are different customer segments and PC1, PC2 are reduced features to 2 dimesions

In [ ]:
plot_colormap(train_x_values, train_y_values, classifier,'Train set')

In [ ]:
plot_colormap(test_x_values, test_y_values, classifier,'Test set')